In [1]:
# add BeautifulSoup for crawl links
from bs4 import BeautifulSoup as bs
# add requests for request to links
import requests
# add pandas for save data's in a DataFrame
import pandas as pd

In [2]:
# find name of laptop by link with this def.
def laptop_name(p_content):
    laptop_name = p_content.find("div",class_="specs_header").get_text().replace("\xa0"," ")
    return laptop_name
# find id of laptop page by link with this def.
def laptop_page_id(link):
    page_id = link.replace("https://www.notebookcheck.net/","")
    return page_id
# find laptop specification by link and add them to a dictionary with this def. (dirty one)
def laptop_spec_dict(p_content):
    laptop_spec = []
    spec_title_list = []
    index_list = []
    laptop_specx = []
    laptop_specy = []
    spec_ = []
    spec_dict = {}
    laptop_spec_unclean = p_content.find("div",class_="specs_whole").get_text("&").replace("\n","").replace("\xa0-\xa0","").replace("\xa0","").split("&")
    laptop_spec_title = p_content.find_all("div",class_="specs")
    for i in range(len(laptop_spec_title)):
        spec_title_list.append(str(laptop_spec_title[i]).split(">",1)[1].replace("</div>",""))
    for i in range(len(laptop_spec_unclean)):
        if laptop_spec_unclean[i] != '' and laptop_spec_unclean[i] != ' ':
            laptop_spec.append(laptop_spec_unclean[i])
    for i in range(len(spec_title_list)):
        if spec_title_list[i] in laptop_spec_unclean:
            index_list.append(laptop_spec.index(spec_title_list[i]))
    for i in range(len(index_list)):
        try:
            y = index_list[i+1]
            laptop_specy.append(y)
            x = index_list[i]
            laptop_specx.append(x)
        except:
            y = index_list[i]
            laptop_specy.append(y)
            x = index_list[i]
            laptop_specx.append(x)
    for i in range(len(laptop_specx)):
        spec_.append(laptop_spec[laptop_specx[i]:laptop_specy[i]])
    spec_.pop(-1)
    for i in range(len(spec_)):
        spec_dict [spec_[i][0]] = spec_[i][1:]
        return spec_dict

In [5]:
# Build DataFrame with "df_notebookcheck" name.
df_notbookcheck = pd.DataFrame(columns=["page_id",
"laptop_name",
"laptop_specifications"])
# make a list of your links and add it here.
list_of_links = []
all_links = open("laptop_links.txt", "r")
for links in all_links:
    list_of_links.append(links.replace("\n",""))
for link in list_of_links:
    p_page = requests.get(link)
    p_content = bs(p_page.content, "html.parser")
    name = laptop_name(p_content)
    p_id = laptop_page_id(link)
    spec_dict = laptop_spec_dict(p_content)
    df_notbookcheck = df_notbookcheck.append({"page_id":p_id,
    "laptop_name":name,"laptop_specifications":spec_dict},ignore_index=True)
df_notbookcheck.to_csv("notebookcheck.csv")